In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from time import sleep
import pandas as pd

chrome_options = Options()
chrome_options.add_experimental_option("detach", True)


In [17]:
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

for season in range(2013, 2023):
  
  
  url = f'https://www.transfermarkt.com/premier-league/chancenverwertung/wettbewerb/GB1/saison_id/{season}/plus/1'
  driver.get(url)

  # 한번만 accept만 하면 된다네요. 이거땜에 두번째부터 이 창이 안나타니 아래 코드가 실행이 안돼서 에러의 원인입니다. 따라서 if season=2013을 넣어야되요
  if season == 2013:
    team_data = []
    # 팝업창 닫기
    iframe = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.XPATH, "//iframe[@id='sp_message_iframe_953358']"))
      )
    driver.switch_to.frame(iframe)
    # "Accept & continue" 버튼에 대한 대기
    accept_button = WebDriverWait(driver, 10).until(
      EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept & continue')]"))
      )
    accept_button.click()
  
  # 자바 스크립트를 활성화한다네요 뭔지 모르지만 chat GPT가 find_element 안될때 실행하라고 알려줬어요
  driver.execute_script("return navigator.userAgent")
  sleep(3)
  
  
  #yw1 > table > tbody > tr:nth-child(1) > td:nth-child(4)
  #yw1 > table > tbody > tr:nth-child(2) > td:nth-child(4)
  
  for i in range(1,21):
    team_name = driver.find_element(By.CSS_SELECTOR, f'#yw1 > table > tbody > tr:nth-child({i}) > td.hauptlink.no-border-links > a').text
    team_total_shot = driver.find_element(By.CSS_SELECTOR, f'#yw1 > table > tbody > tr:nth-child({i}) > td:nth-child(3)').text
    team_valid_shot = driver.find_element(By.CSS_SELECTOR, f'#yw1 > table > tbody > tr:nth-child({i}) > td:nth-child(4)').text
    team_goal_ratio = driver.find_element(By.CSS_SELECTOR, f'  #yw1 > table > tbody > tr:nth-child({i}) > td.rechts.hauptlink').text
    team_data.append({'Team': team_name, 'Total_shots': team_total_shot, 'Valid_shots': team_valid_shot,'Goal_ratio': team_goal_ratio})
    
    
# 데이터프레임 생성
df = pd.DataFrame(team_data)

# CSV 파일로 저장
filename = f'슈팅 데이터.csv'
df.to_csv(filename, index=False)
print(f'{filename} 파일이 생성되었습니다.')
  

슈팅 데이터.csv 파일이 생성되었습니다.


In [18]:
epl_df = pd.read_csv('C:/codingon/Data Analize/Soccer-Data/Final_Data.csv',encoding='euc-kr')
shot_df = pd.read_csv('C:/codingon/Data Analize/Soccer-Data/크롤링 소스/슈팅 데이터.csv',encoding='euc-kr')

In [19]:
shot_df['season'] = epl_df['season']
shot_df

,Team,Total_shots,Valid_shots,Goal_ratio,season
0,Manchester City,458,226,22.3 %,2013-2014
1,Stoke City,205,102,22.0 %,2013-2014
2,Liverpool FC,471,255,21.4 %,2013-2014
3,Arsenal FC,344,191,19.8 %,2013-2014
4,West Bromwich Albion,229,101,18.8 %,2013-2014
...,...,...,...,...,...
195,Manchester United,804,592,7.2 %,2022-2023
196,Everton FC,515,378,6.6 %,2022-2023
197,West Ham United,654,468,6.4 %,2022-2023
198,Chelsea FC,596,433,6.4 %,2022-2023


In [24]:
shot_df_copy = shot_df.groupby('season').apply(lambda group:group.sort_values(by='Team',ascending=True))
shot_df_copy = shot_df_copy.reset_index(level='season',drop=True)
shot_df_copy.head(40)
shot_df_copy.to_csv('C:/codingon/Data Analize/Soccer-Data/크롤링 소스/슈팅 데이터 편집본.csv',encoding='euc-kr',index=False)

In [27]:

epl_df = pd.read_csv('C:/codingon/Data Analize/Soccer-Data/크롤링 소스/슈팅 데이터 편집본.csv',encoding='euc-kr')
# 팀명 한글화
team_eng = ['Arsenal FC', 'Aston Villa', 'Cardiff City', 'Chelsea FC',
       'Crystal Palace', 'Everton FC', 'Fulham FC', 'Hull City',
       'Liverpool FC', 'Manchester City', 'Manchester United',
       'Newcastle United', 'Norwich City', 'Southampton FC', 'Stoke City',
       'Sunderland AFC', 'Swansea City', 'Tottenham Hotspur',
       'West Bromwich Albion', 'West Ham United', 'Burnley FC',
       'Leicester City', 'Queens Park Rangers', 'AFC Bournemouth',
       'Watford FC', 'Middlesbrough FC', 'Brighton & Hove Albion',
       'Huddersfield Town', 'Wolverhampton Wanderers', 'Sheffield United',
       'Leeds United', 'Brentford FC', 'Nottingham Forest']
team_kor = ['아스날','아스톤 빌라','카디프','첼시','크리스탈 팰리스','에버튼','풀럼','헐시티',
            '리버풀','맨시티','맨유','뉴캐슬','노리치','사우스햄튼','스토크시티','선더랜드',
            '스완지','토트넘','웨스트 브로미치','웨스트햄','번리','레스터시티','QPR','본머스',
            '왓포드','미들스브러','브라이튼','허더스필드','울버햄튼','셰필드','리즈','브렌트포드','노팅엄']

for eng in team_eng:
  ab = epl_df.loc[epl_df.Team == eng].index
  idx = []
  for i in ab:
    idx.append(i)
  epl_df.iloc[idx,0] = team_kor[team_eng.index(eng)]
  
epl_df.to_csv('C:/codingon/Data Analize/Soccer-Data/크롤링 소스/슈팅 데이터 편집.csv',encoding='euc-kr',index=False)

In [139]:
shot_df = pd.read_csv('C:/codingon/Data Analize/Soccer-Data/크롤링 소스/슈팅 데이터 편집.csv',encoding='euc-kr')
epl_df = pd.read_csv('C:/codingon/Data Analize/Soccer-Data/Final_Data.csv',encoding='euc-kr')
shot_df

,Team,Total_shots,Valid_shots,Goal_ratio,season
0,아스날,344,191,19.8 %,2013-2014
1,아스톤 빌라,288,135,13.5 %,2013-2014
2,카디프,238,111,13.4 %,2013-2014
3,첼시,452,224,15.7 %,2013-2014
4,크리스탈 팰리스,258,126,12.8 %,2013-2014
...,...,...,...,...,...
195,노팅엄,459,323,8.3 %,2022-2023
196,사우스햄튼,503,362,7.2 %,2022-2023
197,토트넘,608,456,11.5 %,2022-2023
198,웨스트햄,654,468,6.4 %,2022-2023


In [140]:
epl_df['Total_shots'] = shot_df['Total_shots']
epl_df['Valid_shots'] = shot_df['Valid_shots']
epl_df['Goal_ratio(%)'] = shot_df['Goal_ratio']
epl_df['Valid_shot_ratio(%)'] = shot_df['Valid_shots'] / shot_df['Total_shots']
epl_df['Shot_accuracy(%)'] = epl_df['Score']/shot_df['Valid_shots']
epl_df.head()

,Team,Rank,Point,Score,Lose_score,Balance(m),AverageAge,season,goal_gain_lose_gap,Total_shots,Valid_shots,Goal_ratio(%),Valid_shot_ratio(%),Shot_accuracy(%)
0,아스날,4,79,68,41,37.10,26.5,2013-2014,27,344,191,19.8 %,0.555233,0.356021
1,아스톤 빌라,15,38,39,61,11.74,24.4,2013-2014,-22,288,135,13.5 %,0.468750,0.288889
2,카디프,20,30,32,74,42.08,25.2,2013-2014,-42,238,111,13.4 %,0.466387,0.288288
3,첼시,3,82,71,27,52.42,27.3,2013-2014,44,452,224,15.7 %,0.495575,0.316964
4,크리스탈 팰리스,11,45,33,48,33.00,27.7,2013-2014,-15,258,126,12.8 %,0.488372,0.261905


In [141]:
epl_df['Goal_ratio(%)'] = epl_df['Goal_ratio(%)'].map(lambda p : float(p[:-1]))
epl_df['Valid_shot_ratio(%)'] = epl_df['Valid_shot_ratio(%)'].map(lambda p : p*100)
epl_df['Shot_accuracy(%)'] = epl_df['Shot_accuracy(%)'].map(lambda p : p*100)
epl_df

,Team,Rank,Point,Score,Lose_score,Balance(m),AverageAge,season,goal_gain_lose_gap,Total_shots,Valid_shots,Goal_ratio(%),Valid_shot_ratio(%),Shot_accuracy(%)
0,아스날,4,79,68,41,37.10,26.5,2013-2014,27,344,191,19.8,55.523256,35.602094
1,아스톤 빌라,15,38,39,61,11.74,24.4,2013-2014,-22,288,135,13.5,46.875000,28.888889
2,카디프,20,30,32,74,42.08,25.2,2013-2014,-42,238,111,13.4,46.638655,28.828829
3,첼시,3,82,71,27,52.42,27.3,2013-2014,44,452,224,15.7,49.557522,31.696429
4,크리스탈 팰리스,11,45,33,48,33.00,27.7,2013-2014,-15,258,126,12.8,48.837209,26.190476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,노팅엄,16,38,38,68,190.25,26.5,2022-2023,-30,459,323,8.3,70.370370,11.764706
196,사우스햄튼,20,25,36,73,146.61,24.6,2022-2023,-37,503,362,7.2,71.968191,9.944751
197,토트넘,8,60,70,63,139.15,27.1,2022-2023,7,608,456,11.5,75.000000,15.350877
198,웨스트햄,14,40,42,55,164.95,27.9,2022-2023,-13,654,468,6.4,71.559633,8.974359


In [142]:

epl_df_copy = epl_df

epl_df_copy.to_csv('Final_Data_update.csv',encoding='euc-kr',index=False)